In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from collections import Counter
#stopwords = set(STOPWORDS)
import itertools
import nltk
# nltk.download()
# nltk.download('stopwords')
# nltk.download("punkt")
# nltk.download('corpora')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk import ne_chunk
from nltk.util import ngrams
from nltk.book import *



import plotly
import plotly.express as px

import re
import string



#read the files
train = pd.read_csv("/Users/legfot/Documents/content/data/train.csv", encoding = 'ISO-8859-1')
test = pd.read_csv("/Users/legfot/Documents/content/data/test.csv", encoding = 'ISO-8859-1')
test_y = pd.read_csv("/Users/legfot/Documents/content/data/test_labels.csv", encoding = 'ISO-8859-1')

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [4]:

stop = stopwords.words('english')

#removes \n
train['preprocess'] = train.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1) 
test['preprocess'] = test.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1)

#removes urls
train['preprocess']=train.apply(lambda row: re.sub('http://\S+|https://\S+', 'urls',row['preprocess']).lower(), axis=1)
test['preprocess']=test.apply(lambda row: re.sub('http://\S+|https://\S+', 'urls',row['preprocess']).lower(), axis=1)

#remove all non-alphanumeric values(Except single quotes)
train['preprocess']=train.apply(lambda row: re.sub('[^A-Za-z\' ]+', '',row['preprocess']).lower(), axis=1)
test['preprocess']=test.apply(lambda row: re.sub('[^A-Za-z\' ]+', '',row['preprocess']).lower(), axis=1)

#remove stopwords as they occupy major chunk of the vocabulary
train['preprocess'] = train['preprocess'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test['preprocess'] = test['preprocess'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#removes all additional spaces
train['preprocess']=train.apply(lambda row: re.sub('  +', ' ',row['preprocess']).strip(), axis=1)
test['preprocess']=test.apply(lambda row: re.sub('  +', ' ',row['preprocess']).strip(), axis=1)

In [5]:
train.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,preprocess
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits made username hardcore metal...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww matches background colour i'm seemingly ...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man i'm really trying edit war guy constan...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,can't make real suggestions improvement wonder...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,sir hero chance remember page that's


In [6]:
train["preprocess1"] = train.apply(lambda x: x["comment_text"] if len(x["preprocess"])==0 else x['preprocess'], axis=1)
test["preprocess1"] = test.apply(lambda x: x["comment_text"] if len(x["preprocess"])==0 else x['preprocess'], axis=1)
traindf=train[['preprocess1','toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']]
testdf=test[['id','preprocess1']]

In [8]:
traindf.head(5)
# testdf.head(5)

,preprocess1,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,d'aww matches background colour i'm seemingly ...,0,0,0,0,0,0
2,hey man i'm really trying edit war guy constan...,0,0,0,0,0,0
3,can't make real suggestions improvement wonder...,0,0,0,0,0,0
4,sir hero chance remember page that's,0,0,0,0,0,0


In [10]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Input
from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

In [11]:
traind=traindf["preprocess1"]
train_label=traindf[['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']]
testd=testdf["preprocess1"]

#prepare tokenizer
tokenizer = Tokenizer(num_words = 30000) #only 30000 words considered here
tokenizer.fit_on_texts(traind)

#convert each text into array of integers with help of tokenizer.
train_final = tokenizer.texts_to_sequences(traind)
test_final = tokenizer.texts_to_sequences(testd)
#Now lets pad the sentences. 
#From the above histogram it is evident that most of sentences falling in range 1-200 lengths. 
#max length=200 is kept 
#We will trim any sentence above that length and we will pad zeros for all the sentence below 200
traind=pad_sequences(train_final, maxlen=200)
testd=pad_sequences(test_final, maxlen=200)

In [16]:
train_final[1]

[2301, 1253, 3457, 13, 4073, 2369, 20, 4, 789, 83]

In [18]:
train111=traindf["preprocess1"]
train_label=traindf[['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']]
#prepare tokenizer
tokenizer = Tokenizer(num_words = 30000) #only 30000 words considered here
tokenizer.fit_on_texts(train111)


0    explanation edits made username hardcore metal...
1    d'aww matches background colour i'm seemingly ...
2    hey man i'm really trying edit war guy constan...
3    can't make real suggestions improvement wonder...
4                 sir hero chance remember page that's
Name: preprocess1, dtype: object

In [23]:

train1 = tokenizer.fit_on_sequences(train111)
train1[[1]]

TypeError: 'NoneType' object is not subscriptable

In [5]:
train_toxic = train.apply(lambda x : train["toxic"] == 1)
train_toxic

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
159566,False,False,False,False,False,False,False,False
159567,False,False,False,False,False,False,False,False
159568,False,False,False,False,False,False,False,False
159569,False,False,False,False,False,False,False,False


In [8]:
test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."
